[View in Colaboratory](https://colab.research.google.com/github/cahya-wirawan/ML-Collection/blob/master/Text_Classification.ipynb)

In [4]:
!pip install textblob
#!pip install -U ntlk

  Could not find a version that satisfies the requirement ntlk (from versions: )
No matching distribution found for ntlk


In [0]:
from sklearn import model_selection, preprocessing
from sklearn import linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas as pd, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
#import ntlk

In [0]:

#ntlk.download('punkt')


In [0]:
#from google.colab import files

#uploaded = files.upload()

#for fn in uploaded.keys():
#  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

In [7]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [36]:
!pwd
!ls -l "/content/drive/My Drive/lmdata"
#!ls -l "/content/"

/content
total 2210544
-rw------- 1 root root    4507148 Apr 21 23:28 amazon-review.csv
-rw------- 1 root root 2259088777 Mar 14  2018 wiki-news-300d-1M.vec


In [0]:
#df = pd.read_csv('amazon-review.csv')
#drive/My Drive/lmdata/amazon-review.csv

In [0]:
# load the dataset
data = open('drive/My Drive/lmdata/amazon-review.csv').read()
labels, texts = [], []
for i, line in enumerate(data.split("\n")):
    content = line.split(" ", 1)
    labels.append(content[0])
    texts.append(content[1])

# create a dataframe using texts and lables
trainDF = pd.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels


In [38]:
print(trainDF.head())
print(texts[:2])

                                                text       label
0  Stuning even for the non-gamer: This sound tra...  __label__2
1  The best soundtrack ever to anything.: I'm rea...  __label__2
2  Amazing!: This soundtrack is my favorite music...  __label__2
3  Excellent Soundtrack: I truly like this soundt...  __label__2
4  Remember, Pull Your Jaw Off The Floor After He...  __label__2
['Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^', "The best soundtrack ever to anything.: I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This i

In [0]:
!pip show tensorflow

In [0]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [40]:
print(train_x[:5])
print(valid_x[:5])


3877    A stimulating toy but a little too heavy and h...
1609    Seriously flawed AI and interface killed the g...
8481    Would've taken me 100 years to read: I read a ...
2608    Beauty is an Enemy to the less fortunate: Beca...
9677    Hound of the Baskervilles: This edition is mis...
Name: text, dtype: object
2809    Failed to be powerful...or even interesting: O...
9074    Tedious Reds under the beds stuff: Mr Drury's ...
3015    Can't believe I gave this 2 stars: I'm a huge ...
2334    Chris LaMach: Chris LaMach. Chris was my next ...
1818    Expensive - But May Be Worth the Cost: For yea...
Name: text, dtype: object


In [0]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [0]:
#print(xtrain_count, xvalid_count)


In [0]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', 
                             max_features=5000)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', 
                                   ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}',
                                         ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

In [0]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('drive/My Drive/lmdata/wiki-news-300d-1M.vec')):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [0]:
trainDF['char_count'] = trainDF['text'].apply(len)
trainDF['word_count'] = trainDF['text'].apply(lambda x: len(x.split()))
trainDF['word_density'] = trainDF['char_count'] / (trainDF['word_count']+1)
trainDF['punctuation_count'] = trainDF['text'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
trainDF['title_word_count'] = trainDF['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
trainDF['upper_case_word_count'] = trainDF['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [45]:
print(trainDF.head())

                                                text       label  char_count  \
0  Stuning even for the non-gamer: This sound tra...  __label__2         426   
1  The best soundtrack ever to anything.: I'm rea...  __label__2         509   
2  Amazing!: This soundtrack is my favorite music...  __label__2         760   
3  Excellent Soundtrack: I truly like this soundt...  __label__2         743   
4  Remember, Pull Your Jaw Off The Floor After He...  __label__2         481   

   word_count  word_density  punctuation_count  title_word_count  \
0          80      5.259259                 11                10   
1          97      5.193878                 14                 7   
2         129      5.846154                 40                24   
3         118      6.243697                 33                52   
4          87      5.465909                 22                30   

   upper_case_word_count  
0                      3  
1                      3  
2                      4  
3 

In [11]:
#!pip install -U nltk
#import ntlk
#ntlk.download('punkt')

Requirement already up-to-date: nltk in /usr/local/lib/python3.6/dist-packages (3.3)


ModuleNotFoundError: ignored

In [0]:
"""
pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# function to check and get the part of speech tag count of a words in a given sentence
def check_pos_tag(x, flag):
    cnt = 0
    try:
        wiki = textblob.TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
    except:
        pass
    return cnt

trainDF['noun_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'noun'))
trainDF['verb_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'verb'))
trainDF['adj_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'adj'))
trainDF['adv_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'adv'))
trainDF['pron_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'pron'))
"""

In [0]:
# train a LDA Model
lda_model = decomposition.LatentDirichletAllocation(n_components=20, learning_method='online', max_iter=20)
X_topics = lda_model.fit_transform(xtrain_count)
topic_word = lda_model.components_ 
vocab = count_vect.get_feature_names()

# view the topic models
n_top_words = 10
topic_summaries = []
for i, topic_dist in enumerate(topic_word):
    topic_words = numpy.array(vocab)[numpy.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))

In [0]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, 
                is_neural_net=False, epochs=1):
    # fit the training dataset on the classifier
    if is_neural_net:
      classifier.fit(feature_vector_train, label, epochs=epochs)
    else:
      classifier.fit(feature_vector_train, label)   
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    
    if is_neural_net:
      predictions = [int(round(p[0])) for p in predictions]
      #predictions = predictions.argmax(axis=-1)

    print("predictions", predictions[:20])
    print("valid_y", valid_y[:20])
    
    return metrics.accuracy_score(predictions, valid_y)

In [91]:
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print("NB, Count Vectors: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("NB, CharLevel Vectors: ", accuracy)

predictions [1 0 0 1 1 0 1 1 0 0 0 1 1 0 1 0 1 0 0 1]
valid_y [0 0 0 1 1 0 1 1 0 0 1 1 1 0 0 0 1 0 0 1]
NB, Count Vectors:  0.8432
predictions [1 0 0 1 1 0 1 1 0 0 0 1 1 0 1 0 1 0 0 1]
valid_y [0 0 0 1 1 0 1 1 0 0 1 1 1 0 0 0 1 0 0 1]
NB, WordLevel TF-IDF:  0.85
predictions [0 1 0 1 1 1 1 1 0 0 0 1 1 0 0 1 0 0 0 1]
valid_y [0 0 0 1 1 0 1 1 0 0 1 1 1 0 0 0 1 0 0 1]
NB, N-Gram Vectors:  0.8396
predictions [1 0 0 1 1 0 1 1 0 0 1 1 1 0 1 0 1 0 0 1]
valid_y [0 0 0 1 1 0 1 1 0 0 1 1 1 0 0 0 1 0 0 1]
NB, CharLevel Vectors:  0.8072


In [49]:
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
print("LR, Count Vectors: ", accuracy)

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print("LR, WordLevel TF-IDF: ", accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print( "LR, N-Gram Vectors: ", accuracy)

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("LR, CharLevel Vectors: ", accuracy)

LR, Count Vectors:  0.8612
LR, WordLevel TF-IDF:  0.872
LR, N-Gram Vectors:  0.834
LR, CharLevel Vectors:  0.8348


In [63]:
# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("SVM, N-Gram Vectors: ", accuracy)


SVM, N-Gram Vectors:  0.5152


In [51]:
# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print("RF, Count Vectors: ", accuracy)

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print("RF, WordLevel TF-IDF: ", accuracy)

RF, Count Vectors:  0.7416
RF, WordLevel TF-IDF:  0.7552


In [52]:
# Extereme Gradient Boosting on Count Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, xvalid_count.tocsc())
print("Xgb, Count Vectors: ", accuracy)

# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
print("Xgb, WordLevel TF-IDF: ", accuracy)

# Extereme Gradient Boosting on Character Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), train_y, xvalid_tfidf_ngram_chars.tocsc())
print("Xgb, CharLevel Vectors: ", accuracy)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Xgb, Count Vectors:  0.8092


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Xgb, WordLevel TF-IDF:  0.8092
Xgb, CharLevel Vectors:  0.8028


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [109]:
def create_model_architecture(input_size):
    # create input layer 
    input_layer = layers.Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(100, activation="relu")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    return classifier 

classifier = create_model_architecture(xtrain_tfidf_ngram.shape[1])
accuracy = train_model(classifier, xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, 
                       is_neural_net=True, epochs=10)
print("NN, Ngram Level TF IDF Vectors",  accuracy)

Epoch 1/10
7500/7500 [==============================] - 4s 536us/step - loss: 0.5208
Epoch 2/10
7500/7500 [==============================] - 2s 277us/step - loss: 0.2787
Epoch 3/10
7500/7500 [==============================] - 2s 278us/step - loss: 0.1944
Epoch 4/10
7500/7500 [==============================] - 2s 282us/step - loss: 0.1439
Epoch 5/10
7500/7500 [==============================] - 2s 272us/step - loss: 0.1066
Epoch 6/10
7500/7500 [==============================] - 2s 284us/step - loss: 0.0754
Epoch 7/10
7500/7500 [==============================] - 2s 276us/step - loss: 0.0535
Epoch 8/10
7500/7500 [==============================] - 2s 269us/step - loss: 0.0368
Epoch 9/10
7500/7500 [==============================] - 2s 279us/step - loss: 0.0258
Epoch 10/10
7500/7500 [==============================] - 2s 249us/step - loss: 0.0190
predictions [0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1]
valid_y [0 0 0 1 1 0 1 1 0 0 1 1 1 0 0 0 1 0 0 1]
NN, Ngram Level TF IDF Vec

In [112]:
def create_cnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_cnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, 
                       is_neural_net=True, epochs=10)
print("CNN, Word Embeddings",  accuracy)

Epoch 1/10
7500/7500 [==============================] - 5s 664us/step - loss: 0.5851
Epoch 2/10
7500/7500 [==============================] - 3s 342us/step - loss: 0.3899
Epoch 3/10
7500/7500 [==============================] - 3s 344us/step - loss: 0.3265
Epoch 4/10
7500/7500 [==============================] - 3s 342us/step - loss: 0.2747
Epoch 5/10
7500/7500 [==============================] - 3s 347us/step - loss: 0.2305
Epoch 6/10
7500/7500 [==============================] - 3s 350us/step - loss: 0.1895
Epoch 7/10
7500/7500 [==============================] - 3s 347us/step - loss: 0.1605
Epoch 8/10
7500/7500 [==============================] - 3s 338us/step - loss: 0.1172
Epoch 9/10
7500/7500 [==============================] - 3s 347us/step - loss: 0.1063
Epoch 10/10
7500/7500 [==============================] - 3s 352us/step - loss: 0.0929
predictions [1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1]
valid_y [0 0 0 1 1 0 1 1 0 0 1 1 1 0 0 0 1 0 0 1]
CNN, Word Embeddings 0.847

In [114]:
def create_rnn_lstm():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.LSTM(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rnn_lstm()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, 
                       is_neural_net=True, epochs=10)
print("RNN-LSTM, Word Embeddings",  accuracy)

Epoch 1/10
7500/7500 [==============================] - 54s 7ms/step - loss: 0.5859
Epoch 2/10
7500/7500 [==============================] - 51s 7ms/step - loss: 0.4830
Epoch 3/10
7500/7500 [==============================] - 49s 7ms/step - loss: 0.4649
Epoch 4/10
7500/7500 [==============================] - 50s 7ms/step - loss: 0.4363
Epoch 5/10
7500/7500 [==============================] - 50s 7ms/step - loss: 0.4121
Epoch 6/10
7500/7500 [==============================] - 49s 7ms/step - loss: 0.3980
Epoch 7/10
7500/7500 [==============================] - 50s 7ms/step - loss: 0.3667
Epoch 8/10
7500/7500 [==============================] - 50s 7ms/step - loss: 0.3493
Epoch 9/10
7500/7500 [==============================] - 49s 7ms/step - loss: 0.3378
Epoch 10/10
7500/7500 [==============================] - 48s 6ms/step - loss: 0.3246
predictions [0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1]
valid_y [0 0 0 1 1 0 1 1 0 0 1 1 1 0 0 0 1 0 0 1]
RNN-LSTM, Word Embeddings 0.844


In [120]:
%%time

def create_rnn_gru():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the GRU Layer
    lstm_layer = layers.GRU(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rnn_gru()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, 
                       is_neural_net=True, epochs=10)
print("RNN-GRU, Word Embeddings",  accuracy)

Epoch 1/10
7500/7500 [==============================] - 45s 6ms/step - loss: 0.5874
Epoch 2/10
7500/7500 [==============================] - 41s 5ms/step - loss: 0.4638
Epoch 3/10
7500/7500 [==============================] - 41s 5ms/step - loss: 0.4025
Epoch 4/10
7500/7500 [==============================] - 41s 5ms/step - loss: 0.3732
Epoch 5/10
7500/7500 [==============================] - 40s 5ms/step - loss: 0.3440
Epoch 6/10
7500/7500 [==============================] - 41s 5ms/step - loss: 0.3273
Epoch 7/10
7500/7500 [==============================] - 41s 5ms/step - loss: 0.3110
Epoch 8/10
7500/7500 [==============================] - 41s 5ms/step - loss: 0.3070
Epoch 9/10
7500/7500 [==============================] - 40s 5ms/step - loss: 0.2859
Epoch 10/10
7500/7500 [==============================] - 42s 6ms/step - loss: 0.2755
predictions [0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1]
valid_y [0 0 0 1 1 0 1 1 0 0 1 1 1 0 0 0 1 0 0 1]
RNN-GRU, Word Embeddings 0.8532
CPU 

In [0]:
%%time

def create_bidirectional_rnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.Bidirectional(layers.GRU(100))(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_bidirectional_rnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, 
                       is_neural_net=True, epochs=1)
print("RNN-Bidirectional, Word Embeddings",  accuracy)

In [0]:
%%time

def create_rcnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
    
    # Add the recurrent layer
    rnn_layer = layers.Bidirectional(layers.GRU(50, return_sequences=True))(embedding_layer)
    
    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rcnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, 
                       is_neural_net=True, epochs=1)
print("CNN, Word Embeddings",  accuracy)

# New Section